# Numerical Solutions to the Advection Eqaution in 1-D

## Mathematical Problem

The homogenous advection equation in one dimmension is  
\begin{align}
  \frac{\partial u}{\partial t} = a \frac{\partial u}{\partial x}
\end{align}
where $u(x,t)$ is a scalar field (_e.g._ density, enthalpy) and $a$ is the advection velocity. This hyperbolic equation is simple very useful for insight on how to numerically solve hyperbolic PDEs. In order to solve this equation (numerically or analytically) we need inital conidtions at $t_0$  
\begin{align}
  u(x,t_0) = \eta(x)
\end{align}
and boundary conditions  
\begin{align}
\begin{aligned}
  u(0, t) = g_0(t) \;\;\; \text{for} \;\;\; t>0\\
  u(L, t) = g_0(t) \;\;\; \text{for} \;\;\; t>0
\end{aligned}
\end{align}  
on the domain $0<x<L$.

## Numerical Approaches
We will aplly finite differences on a discrete grid with grid points $(x_i, t_j)$ where  
\begin{align}
  x_i = i h && t_j = jk
\end{align}
Here $h = \Delta x$ is the grid cell spacing and $k = \Delta t$ is the time step, with $U^i_j \approx u(x_i, t_j)$ is numerical approximation at $(x_i, t_j)$.  

__Foward-Time Cenetered-Space (FTCS)__  

A first order approach, using centered differences in space 
\begin{align}  
  \frac{U_i^{j+1} - U_i^{j}}{\Delta t} = -\frac{a}{2 \Delta x} \left( U_{i+1}^{j} + U_{i-1}^{j}\right)
\end{align}  
and forward differences in time, can be written as   
\begin{align}  
  U_i^{j+1} = U_i^{j} -\frac{a \Delta t}{2 \Delta x} \left( U_{i+1}^{j} + U_{i-1}^{j}\right) . 
\end{align} 
This method is not useful because of stability limitations and is only first order accurate. We will not use this in practice, but usefull for comparison to more accurate and stable methods.  

__Lax-Wendroff__   
This method is based on the Taylor series expansion  
\begin{align}
  u(x,t_{j+1}) = u(x,t_{j}) + \Delta u_t(x,t_n) + \frac12 (\Delta t)^2 u_{tt}(x,t_n) + \ldots.
  \label{eqn:taylor}
\end{align}  
From the governing equation we have $u_t = a u_x$, differentiating this with respect to $t$ gives  
\begin{align}  
  u_{tt} = -au_{xt} = a^2 u_{xx}
\end{align}
where we have used $u_{xt} = u_{tx} = (a u_x)_x $ based on equality of mixed derivatives. Subing these expressions for $u_t$ and $u_tt$ into Eqn. \ref{eqn:taylor} gives  
\begin{align}
  u(x,t_{j+1}) = u(x,t_{j}) + \Delta t \; au_x(x,t_n) + \frac12 (\Delta t)^2 a^2 u_{xx}(x,t_n) + \ldots.
\end{align}
Using only these first three terms and expanding the spatial derivatives using centered differences gives
\begin{align}  
  U_i^{j+1} = U_i^{j} -\frac{ \Delta t}{2 \Delta x} a \left( U_{i+1}^{j} + U_{i-1}^{j}\right) + \frac{(\Delta t)^2}{2 (\Delta x)^2} a^2 \left(U_{i-1}^{j} -2U_{i}^{j} + U_{i+1}^{j}\right). 
\end{align} 
the _Lax-Wendroff_ method, which is second order accurate. 


__Upwind__  
The above methods are symetric in space, but the advection equation is aysymettric depending on the sign of $a$; if $a > 0$ the solution moves to the right and if $a<0$ the solution moves to the left. Therefore, under special circumstances when the solution is asymetteric we can take advantage of this  
\begin{align}  
  U_i^{j+1} = U_i^{j} -\frac{a \Delta t}{\Delta x} \left( U_{i}^{j} - U_{i-1}^{j}\right) 
\end{align} 
or   
\begin{align}  
  U_i^{j+1} = U_i^{j} -\frac{a \Delta t}{\Delta x} \left( U_{i+1}^{j} - U_{i}^{j}\right) 
\end{align} 

__Beam-Splitting__  
\begin{align}  
  U_i^{j+1} = U_i^{j} -\frac{ \Delta t}{2 \Delta x} a \left( 3U_{i}^{j} - 4U_{i-1}^{j} + U_{i-2}^{j}\right) + \frac{(\Delta t)^2}{2 (\Delta x)^2} a^2 \left(U_{i}^{j} -2U_{i-1}^{j} + U_{i-2}^{j}\right). 
\end{align} 


## Python Implementation

In [1]:
# Global
import sys 
import numpy as np 
import scipy.linalg as LA
import matplotlib.pyplot as plt

# Local
sys.path.append('../')
from advdiff.solvers import TDMA
from advdiff.plot import animation

In [2]:
def init_cond(x):
    return  5*np.sin((x-np.pi/2))+5 

def analytical(x,t):
    return 5*np.sin(((x-(a*t))-np.pi/2))+5 

In [3]:
from advdiff.model import FTCS,UpWind,LaxWendroff,BeamWarming

nitt = 10
a = 3.                    # wave speed 
L = 2*np.pi               # Domain Length 
σ = .5                    # courant number
M = 10
N = 10

err = np.zeros(nitt)
dt  = np.zeros(nitt)
dx  = np.zeros(nitt)

for index in range(nitt):
    dx[index] = L / (N-1)
    dt[index] = 3*(σ*dx[index])**2/a
    x  = np.linspace(dx[index],L,M)  # spatial grid
    u  = np.zeros((M,N+1))    # (num methods) X (nx) X (nt)
    exact = np.zeros((M,N+1))
    u[:,0] = init_cond(x)     # init. condition
    
    for t in range(0,N):
        exact[:,t] = analytical(x,dt[index]*t)
    
    u  = UpWind(u,N,dt[index],dx[index],a)
    err[index] = LA.norm(u - exact, 2)
    N *= 2
    M *= 2

# fig, ax = plt.subplots(1,1)
# ax.set_yscale('log')
# ax.set_xscale('log')

# ax.plot(np.log10((L / dx ) + 1),np.log10(err))

ImportError: cannot import name 'FTCS'

In [ ]:
fig, ax = plt.subplots(1,1)

ax.set_yscale('log')
ax.set_xscale('log')

ax.plot((1/dt)+1, err, 'x-')

In [ ]:
(L/dt)+1

In [ ]:
import numpy as np
from advdiff.model import model

params = {'L':2*np.pi,'nx':100,'T':2,'nt':100}
test = model(**params)

In [ ]:
from advdiff.model import model


In [ ]:
########################################################
#################   Init. Constant   ###################
########################################################
a  = 3.                    # wave speed 
L  = 2*np.pi               # Domain Length 
nx = 50                    # Num. grid cells
dx = L/(nx-1)              # grid spacing

nt = 1000                  # Num time steps
σ  = .5                    # courant number
dt = 0.001                 # time step 

########################################################
##################   Init. Domain   ####################
########################################################
x  = np.linspace(dx,L,nx)  # spatial grid
u  = np.zeros((5,nx,nt+1)) # (num methods) X (nx) X (nt)
u[:,:,0] = analytical(x,0)    # init. condition

for j in range(0,nt):
    # FTCS (0)
    u[0,1:-1,j+1] = u[0,1:-1,j] - ((dt*a)/(dx*2)) * (u[0,2:,j] - u[0,0:-2,j])
    # Up-wind (1)
    u[1,1:-1,j+1] = u[1,1:-1,j] - ((dt*a)/dx)* (u[1,1:-1,j] - u[1,0:-2,j])  
    # Lax-Wendroff (2)
    u[2,1:-1,j+1] = u[2,1:-1,j] - ((dt*a)/(dx*2))*(u[2,2:,j] - u[2,0:-2,j]) + ((dt*a)**2/(2*dx**2))*(u[2,2:,j] - 2*u[2,1:-1,j] + u[2,0:-2,j])
    
    for i in range(1,nx-1):  
        # Beam-Splitting (3)
        u[3,i,j+1] = u[3,i,j] - ((dt*a)/(dx*2))* (3*u[3,i,j] - 4*u[3,i-1,j] + u[3,i-2,j]) + ((dt*a)**2/(2*dx**2))*(u[3,i,j] - 2*u[3,i-1,j] + u[3,i-2,j])
        
    #Anlytical (4)
    u[4,:,j+1] = analytical(x,(j+1)*dt)
        
    # Periodic Boundary Cond.
    u[:4,0,j+1]  = u[:4,-2,j+1]
    u[:4,-1,j+1] = u[:4,1,j+1]

In [ ]:
# for j in range(0,nt):
#     #plt.plot(x,u[0,:,j],label='FTCS')
plt.plot(x,u[0,:,j],'x-',label='FTCS')
plt.plot(x,u[4,:,j],'-',label='FTCS')

In [ ]:
LA.norm(u[0,:,:] - u[4,:,:])

In [ ]:
error =np.zeros((4,nx,nt+1))

# FTCS (0)
error[0,:,:] = u[0,:,:] - u[4,:,:]
# Up-wind (1)
error[1,:,:] = u[1,:,:] - u[4,:,:]
# Lax-Wendroff (2)
error[2,:,:] = u[2,:,:] - u[4,:,:]
# Beam-Splitting (3)
error[3,:,:] = u[3,:,:] - u[4,:,:]

for j in range(0,nt):
    plt.plot(x,u[0,:,j],label='FTCS')

In [ ]:
plt.imshow(u[0,:,:])

In [ ]:
plt.plot(init_cond(x))

In [ ]:
u[4,:,1]

In [ ]:
u[0,1:-1,j].shape

In [ ]:
test = np.arange(0,10)

In [ ]:
test[-1]